# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.123600e+02     1.548024e+00
 * time: 0.07417988777160645
     1     1.071699e+01     1.098654e+00
 * time: 1.4080510139465332
     2    -1.091985e+01     1.245442e+00
 * time: 1.528843879699707
     3    -3.400914e+01     8.379143e-01
 * time: 1.6254680156707764
     4    -4.718181e+01     5.836052e-01
 * time: 1.7334299087524414
     5    -5.677748e+01     2.088449e-01
 * time: 1.8455729484558105
     6    -5.968484e+01     1.581053e-01
 * time: 1.9364080429077148
     7    -6.088269e+01     6.328205e-02
 * time: 2.0105319023132324
     8    -6.138085e+01     4.056628e-02
 * time: 2.084514856338501
     9    -6.164766e+01     3.466255e-02
 * time: 2.1625468730926514
    10    -6.181741e+01     2.986695e-02
 * time: 2.2508440017700195
    11    -6.196895e+01     2.251096e-02
 * time: 2.324172019958496
    12    -6.206897e+01     1.901357e-02
 * time: 2.3991198539733887
    13    -6.211259e+01     1.606290e-02
 * time: 2.476223945617

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671059
    AtomicLocal         -18.8557670
    AtomicNonlocal      14.8522651
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485363 
    Xc                  -19.3336817

    total               -62.261666456866
